In [ ]:
from mne.io import read_raw_fif
from brainpipe.feature import power
import numpy as np
import neurokit
import time
import matplotlib.pyplot as plt

In [ ]:
# 1.
# Fetch time series data from any file whatsoever
# Must be at least 30 seconds at 250Hz

raw = read_raw_fif('/home/hyruuk/GitHub/eegsynth/datafiles/SA04_01_preprocessed.fif.gz', preload=True)
raw_rs = raw.copy().resample(250)
sf = raw_rs.info['sfreq']

eeg_timeseries = np.asarray(raw_rs.get_data()[1:5,7500:15000])

In [ ]:
# 2.
# Compute hilbert using brainpipe
def get_hilbert(data, freq=1):
    f = [[4, 8], [8, 12], [12, 30]]
    print(data.shape)
    power_obj = power(sf, npts=data.shape, f=f, baseline=None, norm=None, method='hilbert', window=None, width=None, step=None, split=None, time=None)
    start = time.time()
    power_vals = power_obj.get(data.T)[0].squeeze() # get power values across time
    stop = time.time()
    print(stop-start)
    print(power_vals.shape)
    print('Done')
    #powow = power_vals[0].squeeze()
    dtp = np.average(power_vals[freq,:,:], axis=1) # average sensors and retain the desired freq << must be optimized
    t = np.arange(0,len(dtp),1)
    plt.plot(t, dtp)
    plt.show()
    return dtp

dtp = get_hilbert(eeg_timeseries, freq=1)


In [ ]:
# 3.
# Compute complexity using neurokit
start = time.time()
comp = neurokit.complexity(eeg_timeseries)
stop = time.time()
print("duration of computation :")
print(stop-start)
print(comp)


In [ ]:
# Let's do the same with simulated data ! 

import numpy as np

fs = 250
fund_freq = 10
dur = 30
samples = np.linspace(0, dur, dur * fs)
print(samples)
pure = np.sin(2 * np.pi * fund_freq * samples)
noise = np.random.normal(0, 1, pure.shape)
sim_signal = pure + noise

plt.plot(pure)
plt.show()
plt.plot(noise)
plt.show()
plt.plot(sim_signal)
plt.show()

def comp_compute(data, name):
    start = time.time()
    comp = neurokit.complexity(data)
    stop = time.time()

    print('Complexity of {}'.format(name))
    print("Computed in {} seconds".format(stop-start))
    print(comp)
    return comp


eeg_freqenv = get_hilbert(eeg_timeseries, freq=2)
comp = []
names = ['pure', 'noise', 'sim_signal', 'eeg_freqenv']
for i, data in enumerate([pure, noise, sim_signal, eeg_freqenv]):
    comp.append(comp_compute(data, names[i]))
    